In [1]:
import matplotlib.pyplot as plt
from matplotlib  import rc
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import plotly.express as px
from plotly import graph_objs as go


## Load bank-tract + census data

In [2]:
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

#denial_rate, credit_spread, np.log(# loans), np.log($ loans), avg LTV 
bank_tract ["white_rate"]        = (bank_tract ["WhitePop"]) / bank_tract ["Tot.Pop"] * 100
bank_tract ['majority-minority'] = bank_tract ["white_rate"] < 50
bank_tract ['below_p10_income']  = bank_tract ["Med.HousehIncome"] < bank_tract["Med.HousehIncome"].quantile(0.1)
bank_tract ['high_hispanic']     = (bank_tract ['HispanicLatinoPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_black']        = (bank_tract ['BlackPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_asian']        = (bank_tract ['AsianPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_white']        = (bank_tract ['WhitePop'] / bank_tract['Tot.Pop']) > 0.25

bank_tract ['log(numloans)']     = np.log(bank_tract['num_approved_loans'])
bank_tract ['log(dol_loans)']    = np.log(bank_tract['avg_approved_loan_size'])

In [3]:
# reduce to focal counties (hopefully makes the maps smaller files)
bank_tract = bank_tract.query('(state == 4 & county in [13,19]) | (state == 6 & county in [37,75, 81, 1,85]) ')

## Need to reduce to tract level data. 

Choice here: Just plot competitors. 

Better options avail. Suggestion:
- Piece 1: Reduce bank_tract to the listofshading variables. Then set_index([which_bank, census_tract]), then "unstack" to move which_bank up. 
- Piece 2: Reduce bank_tract to the listofredlining variables. Then drop_duplicates() based on census_tract.
- Merge those two things together. 
- Now it's easy to define more of the variables we've discussed. 
    - log(1+numloans)
    - diff between BOW and Competitors
    - less granular 
    - a new shading variable that is "% of # of loans this bank makes in this tract" should be defined in every tract (0 if missing!)
    - a new shading variable that is "% of $ of loans this bank makes in this tract" should be defined in every tract (0 if missing!)


In [4]:
tract_level = bank_tract.query('which_bank == "All Other Banks"')

## Get the shapefiles

In [5]:
# Get the Arizona tract shapefile and convert to UTM Zone 17N coordinate system
shape_az = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip") #.to_crs(epsg=32617)
shape_ca = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_06_tract.zip") #.to_crs(epsg=32617)
shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)
shape_all['census_tract'] = shape_all['GEOID'].astype(np.int64)

In [6]:
tract_with_shape = shape_all.merge(tract_level, how = 'inner', on = ['census_tract'],
                                   indicator = True, validate= '1:m') 

## Plot

- color for the shapes
- red dots for areas of concern (defined by...)

In [7]:
concerns = tract_with_shape.query('`majority-minority`').copy()
# we need this for the "area of concern" red marks
concerns['lon'] = concerns['INTPTLON'].astype(float)
concerns['lat'] = concerns['INTPTLAT'].astype(float)

In [ ]:
listofshading   = ['denial_rate', 'mean_approved_rate_spread', 'log(numloans)', 'log(dol_loans)', 'mean_LTV']

for v in listofshading:

    fig = px.choropleth_mapbox(tract_with_shape.set_index('GEOID'),
                               geojson=tract_with_shape.geometry,
                               locations=tract_with_shape.index,
                               opacity=.7,
                               color=v, color_continuous_scale=px.colors.sequential.Greens,
                               center={"lat": 33.44, "lon": -112.074036},
                               mapbox_style="open-street-map",
                               zoom=5)

    fig.add_scattermapbox(
        lat = concerns['lat'],
        lon = concerns['lon'],
        mode = 'markers',
        marker_size=12,
        marker_color='red'
    )

    fig.update_layout(
        title=v,
        autosize=False,
        width=1000,
        height=1000,
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    fig.write_html(f"../images/plotly/{v}.html")